c# Comprendre le Positional Encoding (PE)

**Note de synthèse**

---

## Pourquoi le PE est nécessaire ?

Dans un Transformer, tous les mots sont traités **en parallèle** (pas séquentiellement comme un RNN). Sans information de position, le modèle ne distinguerait pas :

- *"Le chat mange la souris"*
- *"La souris mange le chat"*

Les mêmes mots → les mêmes embeddings → même résultat. C'est un problème !

**Solution** : Ajouter un vecteur de position (PE) à chaque embedding.

```
embedding_final = embedding_mot + PE[position]
```

---

## La formule du PE sinusoïdal

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

### Décortiquons chaque terme

| Terme | Signification |
|-------|---------------|
| `PE` | Matrice de taille (seq_len × d_model) |
| `pos` | Position **absolue** du mot dans la séquence (0, 1, 2, ...) |
| `2i` / `2i+1` | Indice de la dimension (colonne) du vecteur |
| `d_model` | Taille des embeddings (ex: 512) |
| `10000` | Constante arbitraire (assez grande pour couvrir de longues séquences) |

### Ce que ça donne concrètement

Pour un mot à la position `pos`, on calcule un vecteur de `d_model` éléments :

```
PE[pos] = [élément_0, élément_1, élément_2, élément_3, ...]
```

- **Élément 0** (dim paire, i=0) : `sin(pos / 10000^(0/d_model))` = `sin(pos / 1)` = `sin(pos)`
- **Élément 1** (dim impaire, i=0) : `cos(pos / 10000^(0/d_model))` = `cos(pos / 1)` = `cos(pos)`
- **Élément 2** (dim paire, i=1) : `sin(pos / 10000^(2/d_model))`
- **Élément 3** (dim impaire, i=1) : `cos(pos / 10000^(2/d_model))`
- etc.

In [ ]:
import torch
import math
import matplotlib.pyplot as plt
import numpy as np

def get_positional_encoding(seq_len, d_model):
    """
    Génère le positional encoding avec la formule sin/cos.
    
    Retourne une matrice (seq_len, d_model) où chaque ligne
    est le vecteur PE pour une position donnée.
    """
    position = torch.arange(seq_len).unsqueeze(1)  # (seq_len, 1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
    )  # (d_model/2,)
    
    pe = torch.zeros(seq_len, d_model)
    pe[:, 0::2] = torch.sin(position * div_term)  # dimensions paires
    pe[:, 1::2] = torch.cos(position * div_term)  # dimensions impaires
    
    return pe

---

## Exemple détaillé : d_model=8, pos=3

Calculons le vecteur PE pour le mot en position 3 avec des embeddings de taille 8.

In [ ]:
d_model = 8
pos = 3

print(f"PE pour position {pos} avec d_model={d_model}")
print("=" * 60)

for dim in range(d_model):
    i = dim // 2  # i pour la formule
    exposant = (2 * i) / d_model
    diviseur = 10000 ** exposant
    argument = pos / diviseur
    
    if dim % 2 == 0:  # dimension paire → sin
        valeur = math.sin(argument)
        formule = f"sin({pos} / 10000^{exposant:.2f})"
        func = "sin"
    else:  # dimension impaire → cos
        valeur = math.cos(argument)
        formule = f"cos({pos} / 10000^{exposant:.2f})"
        func = "cos"
    
    print(f"Dim {dim} | i={i} | {func}({pos}/{diviseur:>7.1f}) = {func}({argument:.4f}) = {valeur:>7.4f}")

In [ ]:
# Vérification avec notre fonction
pe = get_positional_encoding(10, 8)
print(f"\nVecteur PE[{pos}] complet :")
print(pe[pos].numpy().round(4))

---

## Le rôle du diviseur : pourquoi les valeurs varient différemment

Le diviseur `10000^(2i/d_model)` augmente avec l'indice de dimension :

| Dimension | i | Exposant (2i/d_model) | Diviseur |
|-----------|---|----------------------|----------|
| 0-1 | 0 | 0 | 1 |
| 2-3 | 1 | 0.25 | 10 |
| 4-5 | 2 | 0.5 | 100 |
| 6-7 | 3 | 0.75 | 1000 |

**Conséquence** : l'argument de sin/cos devient de plus en plus petit pour les dimensions hautes.

- Dimensions basses → `sin(pos)` → varie beaucoup entre positions
- Dimensions hautes → `sin(pos/1000)` → varie peu entre positions

In [ ]:
# Comparaison : comment chaque dimension varie selon la position
print("Dimension 0 (diviseur=1) vs Dimension 6 (diviseur=1000)")
print("=" * 55)
print(f"{'pos':<5} {'dim 0 (sin(pos/1))':<22} {'dim 6 (sin(pos/1000))':<22}")
print("-" * 55)

for pos in [0, 1, 2, 3, 10, 100]:
    val_dim0 = math.sin(pos / 1)
    val_dim6 = math.sin(pos / 1000)
    print(f"{pos:<5} {val_dim0:<22.4f} {val_dim6:<22.6f}")

**Observation** : 
- Dimension 0 : les valeurs changent beaucoup (0 → 0.84 → 0.91 → 0.14...)
- Dimension 6 : les valeurs changent peu (0 → 0.001 → 0.002 → 0.003...)

---

## Pourquoi utiliser sin ET cos ?

Chaque paire (sin, cos) forme un **point sur un cercle** dans l'espace 2D :

```
Position 0 : (sin(0), cos(0)) = (0, 1)       → haut du cercle
Position 1 : (sin(1), cos(1)) = (0.84, 0.54) → tourne...
Position 2 : (sin(2), cos(2)) = (0.91, -0.42)
...
```

C'est comme une **aiguille qui tourne** sur un cadran.

In [ ]:
# Visualisation : le cercle sin/cos pour les dimensions 0-1
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

positions = range(20)

# Cercle pour dimensions 0-1 (diviseur = 1)
ax = axes[0]
sins = [math.sin(p / 1) for p in positions]
coss = [math.cos(p / 1) for p in positions]
ax.scatter(sins, coss, c=positions, cmap='viridis', s=100)
for p in positions:
    ax.annotate(str(p), (sins[p], coss[p]), fontsize=8)
ax.set_xlabel('sin (dim 0)')
ax.set_ylabel('cos (dim 1)')
ax.set_title('Dimensions 0-1 (diviseur=1)\nTourne VITE')
ax.set_xlim(-1.3, 1.3)
ax.set_ylim(-1.3, 1.3)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# Cercle pour dimensions 4-5 (diviseur = 100)
ax = axes[1]
sins = [math.sin(p / 100) for p in positions]
coss = [math.cos(p / 100) for p in positions]
ax.scatter(sins, coss, c=positions, cmap='viridis', s=100)
for p in positions:
    ax.annotate(str(p), (sins[p], coss[p]), fontsize=8)
ax.set_xlabel('sin (dim 4)')
ax.set_ylabel('cos (dim 5)')
ax.set_title('Dimensions 4-5 (diviseur=100)\nTourne LENTEMENT')
ax.set_xlim(-0.3, 0.3)
ax.set_ylim(0.95, 1.01)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Pourquoi des diviseurs différents ? (L'analogie de l'horloge)

C'est comme une horloge avec plusieurs aiguilles :

| Dimensions | Diviseur | Analogie |
|------------|----------|----------|
| 0-1 | 1 | Trotteuse (secondes) - tourne vite |
| 2-3 | 10 | Aiguille des minutes |
| 4-5 | 100 | Aiguille des heures |
| 6-7 | 1000 | Calendrier (jours) - tourne lentement |

**Ensemble**, ces aiguilles créent une **signature unique** pour chaque instant (position).

Pour que deux positions aient le même PE, il faudrait que **TOUTES** les aiguilles soient au même endroit simultanément. C'est quasi-impossible car les périodes ne sont pas des multiples entiers.

---

## À quoi sert le PE dans le Transformer ?

Le PE est ajouté **une seule fois** au début, puis reste mélangé à l'embedding.

```
embedding_final = embedding_mot + PE[position]
```

### Impact sur l'attention

Quand on calcule Q, K, V :

```
Q = W_q × (embedding + PE)    → contient l'info de position
K = W_k × (embedding + PE)    → contient l'info de position

Score = Q · K  → peut prendre en compte "qui est où"
```

**Sans PE** : l'attention ne sait pas si un mot est avant ou après un autre.

**Avec PE** : le modèle peut apprendre des patterns comme :
- "Le sujet est souvent en position 0-1"
- "L'adjectif est souvent juste avant le nom"
- etc.

---

## L'impact de d_model sur le PE

La formule est **conçue pour s'adapter** à n'importe quel d_model.

Grâce à l'exposant `2i/d_model`, le diviseur va toujours de **1 à ~10000**, quelle que soit la taille de l'embedding.

In [ ]:
# Comparaison d_model=64 vs d_model=512
print("Plage des diviseurs selon d_model")
print("=" * 50)

for d_model in [64, 256, 512]:
    div_min = 10000 ** (0 / d_model)
    div_max = 10000 ** ((d_model - 2) / d_model)
    n_frequences = d_model // 2
    print(f"d_model={d_model:3d} : diviseur de {div_min:.0f} à {div_max:.0f} ({n_frequences} paires sin/cos)")

**Analogie** : C'est comme un escalier de même hauteur totale.

```
d_model=64  : ████████ (32 marches hautes)

d_model=512 : ▁▂▃▄▅▆▇█ (256 marches basses)
```

Plus de dimensions = plus de "marches" = variations moins intenses entre dimensions consécutives, mais même couverture globale.

---

## Propriétés importantes du PE sinusoïdal

### 1. Valeurs bornées
Toutes les valeurs sont dans [-1, 1] (propriété de sin/cos).

### 2. Chaque position a un vecteur unique
Grâce aux multiples fréquences combinées.

### 3. Distance constante entre positions consécutives
La distance entre PE(pos) et PE(pos+k) dépend principalement de **k**, pas de **pos**.

In [ ]:
# Vérification : distance entre positions consécutives
pe = get_positional_encoding(1000, 64)

print("Distance entre positions consécutives (PE[pos] vs PE[pos+1])")
print("=" * 50)

for pos in [0, 10, 100, 500, 998]:
    dist = torch.norm(pe[pos] - pe[pos + 1]).item()
    print(f"pos={pos:3d} vs pos={pos+1:3d} : distance = {dist:.4f}")

---

## Visualisation complète du PE

In [ ]:
# Heatmap du PE
pe_visu = get_positional_encoding(100, 64)

plt.figure(figsize=(14, 6))
plt.imshow(pe_visu.T, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
plt.xlabel('Position dans la séquence')
plt.ylabel('Dimension du vecteur')
plt.title('Positional Encoding (100 positions × 64 dimensions)')
plt.colorbar(label='Valeur')

# Annotations
plt.axhline(y=1.5, color='white', linestyle='--', alpha=0.5)
plt.text(50, 0, 'Dimensions 0-1 : oscillation rapide', color='white', ha='center', fontsize=9)
plt.axhline(y=62.5, color='white', linestyle='--', alpha=0.5)
plt.text(50, 63, 'Dimensions 62-63 : oscillation lente', color='white', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---

## Limites du PE sinusoïdal

### 1. Position absolue, pas relative
Le PE encode "je suis en position 5", pas "je suis 2 positions après le verbe". Le modèle doit apprendre les relations relatives à partir des positions absolues.

### 2. Généralisation limitée
Si le modèle est entraîné sur des séquences de 512 tokens, il peut avoir du mal avec des séquences de 2048 tokens (positions jamais vues).

### 3. Pas de sémantique
Le PE ne sait pas que la position du sujet est "importante". C'est le modèle qui doit l'apprendre.

---

## Alternatives modernes au PE sinusoïdal

### 1. Learned Positional Embeddings (BERT, GPT-2)

Au lieu de calculer le PE avec sin/cos, on **apprend** un vecteur pour chaque position.

```python
self.position_embeddings = nn.Embedding(max_position, d_model)
# Matrice apprise de taille (max_position × d_model)
```

**Avantages** :
- Le modèle apprend exactement ce qui est utile
- Simple à implémenter

**Inconvénients** :
- Limité à `max_position` (pas de généralisation au-delà)
- Plus de paramètres

---

### 2. RoPE - Rotary Position Embedding (LLaMA, Mistral)

Au lieu d'**ajouter** le PE à l'embedding, on **tourne** les vecteurs Q et K dans l'espace.

```
Q_rotated = rotate(Q, position)
K_rotated = rotate(K, position)
```

**Idée clé** : Le produit scalaire `Q · K` devient sensible à la **position relative** (pas absolue).

**Avantages** :
- Encode naturellement les positions relatives
- Meilleure généralisation sur les longues séquences
- Utilisé par les modèles les plus performants (LLaMA, Mistral, etc.)

---

### 3. ALiBi - Attention with Linear Biases (BLOOM)

Pas de PE du tout ! On ajoute un **biais** directement aux scores d'attention.

```
Score[i, j] = Q[i] · K[j] - m × |i - j|
```

Plus deux tokens sont éloignés, plus le score est pénalisé.

**Avantages** :
- Très simple
- Excellente généralisation (peut traiter des séquences plus longues que l'entraînement)
- Pas de paramètres supplémentaires

**Inconvénients** :
- Biais vers les tokens proches (peut manquer des dépendances longues)

---

### 4. Relative Position Embeddings (T5, Transformer-XL)

Apprend des embeddings pour les **distances relatives** plutôt que les positions absolues.

```
Score[i, j] = Q[i] · K[j] + position_bias[i - j]
```

**Avantages** :
- Encode directement "2 positions avant/après"
- Meilleure généralisation

**Inconvénients** :
- Plus complexe à implémenter
- Nécessite de limiter la distance max

---

## Tableau récapitulatif

| Méthode | Utilisé par | Position | Généralisation | Complexité |
|---------|-------------|----------|----------------|------------|
| **Sinusoïdal** | Transformer original | Absolue | Moyenne | Simple |
| **Learned** | BERT, GPT-2 | Absolue | Limitée (max_pos) | Simple |
| **RoPE** | LLaMA, Mistral | Relative | Bonne | Moyenne |
| **ALiBi** | BLOOM | Relative | Excellente | Très simple |
| **Relative PE** | T5 | Relative | Bonne | Complexe |

---

## Conclusion

Le PE sinusoïdal est une solution élégante pour encoder la position :

1. **Simple** : juste des sin/cos, pas de paramètres à apprendre
2. **Unique** : chaque position a une signature distincte
3. **Borné** : valeurs entre -1 et 1

Mais les architectures modernes préfèrent des méthodes qui encodent les **positions relatives** (RoPE, ALiBi) car elles généralisent mieux sur les longues séquences.

Le PE est ajouté une fois au début et "disparaît" dans les calculs suivants. C'est le modèle (via W_q, W_k, W_v) qui apprend à exploiter cette information de position.